# NLP Standard Project:

- Students: **Matteo Belletti**, **Alessandro Pasi**, **Stricescu Razvan Ciprian**.

In [22]:
import numpy as np
import matplotlib.pyplot as plt

## Data loading and preprocessing:

In [23]:
# open json in project_data_MELD folder
import json
with open('project_data_MELD/MELD_train_efr.json') as f:
    data = json.load(f)
print(f"Number of samples: {len(data)}")
print(f"Example of a sample: {data[0]}")

Number of samples: 4000
Example of a sample: {'episode': 'utterance_0', 'speakers': ['Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler'], 'emotions': ['neutral', 'neutral', 'neutral', 'neutral', 'surprise'], 'utterances': ["also I was the point person on my company's transition from the KL-5 to GR-6 system.", "You must've had your hands full.", 'That I did. That I did.', "So let's talk a little bit about your duties.", 'My duties?  All right.'], 'triggers': [0.0, 0.0, 0.0, 1.0, 0.0]}


In [24]:
# Convert data to pandas dataframe
import pandas as pd
df = pd.DataFrame(data)
# drop episode and speakers columns
df = df.drop(columns=['episode', 'speakers'])
print(f"Dataframe shape: {df.shape}")
print(f"Dataframe columns: {df.columns}")

Dataframe shape: (4000, 3)
Dataframe columns: Index(['emotions', 'utterances', 'triggers'], dtype='object')


Changing nan values to zeros in order to avoid errors.

In [25]:
# Chech for NaN values on triggers column
nan_counter = 0
for row in df["triggers"]:
    for elem in row:
        if elem != 1 and elem != 0:
            nan_counter += 1

print(f"Number of NaN values changed: {nan_counter}")

Number of NaN values changed: 9


In [26]:
df["triggers"] = df["triggers"].apply(lambda x: [0.0 if elem != 1 and elem != 0 else elem for elem in x])

In [27]:
# I split the dataframe as 80% train, 10% test, 10% validation
from sklearn.model_selection import train_test_split
random_state = 42

train, test = train_test_split(df, test_size=0.2, random_state=random_state)
test, val = train_test_split(test, test_size=0.5, random_state=random_state)

len(train), len(test), len(val)

(3200, 400, 400)

In [47]:
# I import bert-base-uncased
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# I tokenize the sentences
def tokenize_sentences(sentences, tokenizer):
    input_ids = []
    attention_masks = []
    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 64,           
                            padding='max_length',
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',     
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks

# I tokenize the sentences
train_sentences = train["utterances"].values
train_labels = train["triggers"].values
train_input_ids, train_attention_masks = tokenize_sentences(train_sentences, tokenizer)
 
test_sentences = test["utterances"].values
test_labels = test["triggers"].values
test_input_ids, test_attention_masks = tokenize_sentences(test_sentences, tokenizer)

val_sentences = val["utterances"].values
val_labels = val["triggers"].values
val_input_ids, val_attention_masks = tokenize_sentences(val_sentences, tokenizer)

In [48]:
print(train_input_ids.dtype, train_attention_masks.dtype, train_labels.dtype)

torch.int64 torch.int64 object


In [50]:
types = []
for el in train_labels:
    for l in el:
        types.append(type(l))
        if type(l) != float:
            print(l)
print(set(types))

{<class 'float'>}


In [52]:
print(type(train_labels), type(train_input_ids), type(train_attention_masks))

<class 'numpy.ndarray'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [81]:
def prepare_and_pad_labels(labels):
    # Convert the list of lists into a list of tensors
    labels = [torch.tensor(l) for l in labels]
    # Pad the list of tensors to max length
    labels = torch.nn.utils.rnn.pad_sequence(labels, 
                                             batch_first=True, 
                                             padding_value=0.0)
    return labels
# arr = [[elem for elem in el] for el in train_labels]
# # Find the maximum length of sub-lists
# max_len = max(map(len, arr))

# # Create a new list of lists with padding
# padded_data = [lst + [0.0]*(max_len-len(lst)) for lst in arr]

# # Convert the padded list of lists into a tensor
# tensor_data = torch.tensor(padded_data)
prepare_and_pad_labels(train_labels)

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.]]),
 array([list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]),
        list([1.0, 1.0]), list([1.0, 0.0, 1.0]), ...,
        list([0.0, 0.0, 0.0]),
        list([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]),
        list([0.0, 1.0, 1.0, 1.0, 0.0])], dtype=object))

In [83]:
# Create the dataloaders
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataset = TensorDataset(train_input_ids, train_attention_masks, prepare_and_pad_labels(train_labels))
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, prepare_and_pad_labels(test_labels))
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

val_dataset = TensorDataset(val_input_ids, val_attention_masks, prepare_and_pad_labels(val_labels))
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

In [84]:
# I import the model
from transformers import BertForSequenceClassification, AdamW
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# I create the optimizer
optimizer = AdamW(model.parameters(),
                    lr = 2e-5,
                    eps = 1e-8
                    )

# I create the scheduler
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# I create the accuracy function
import numpy as np
from sklearn.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# I create the f1_score function
def flat_f1_score(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)

model.safetensors: 100%|██████████| 440M/440M [01:28<00:00, 5.00MB/s] 
C:\Users\Matteo\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Matteo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequence

In [85]:
# I create the training loop
import random
import numpy as np
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cpu


In [86]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = outputs[0]
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)            
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_f1 = 0
    nb_eval_steps = 0
    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        total_eval_f1 += flat_f1_score(logits, label_ids)
        nb_eval_steps += 1
    avg_val_accuracy = total_eval_accuracy / nb_eval_steps
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_f1 = total_eval_f1 / nb_eval_steps
    print("  F1-Score: {0:.2f}".format(avg_val_f1))


======== Epoch 1 / 4 ========
Training...


ValueError: Target size (torch.Size([32, 24])) must be the same as input size (torch.Size([32, 2]))